In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os


## 1. 初始化设置和数据准备

In [ ]:
# 创建保存图像的文件夹
os.makedirs('res', exist_ok=True)

# 设置超参数和设备
batch_size = 2048
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"使用设备: {device}")
print(f"批大小: {batch_size}")

In [ ]:
# 数据加载
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transform),
    batch_size=batch_size, shuffle=False)

print("训练数据形状:", train_loader.dataset.data.shape)
print("测试数据形状:", test_loader.dataset.data.shape)
print("类别数量: 10 (数字0-9)")

## 2. 数据可视化

In [ ]:
# 显示一些训练样本
def show_samples(loader, num_samples=10):
    data_iter = iter(loader)
    images, labels = next(data_iter)
    
    plt.figure(figsize=(12, 3))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        plt.title(f'Label: {labels[i].item()}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

print("训练样本示例:")
show_samples(train_loader)

## 3. 定义MLP模型

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(784, 128)
        self.l2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

# 创建模型和优化器
model = MLP().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

print("模型结构:")
print(model)

# 计算参数数量
total_params = sum(p.numel() for p in model.parameters())
print(f"\n总参数数量: {total_params:,}")

## 4. 训练模型

In [ ]:
epochs = 10
train_losses = []
train_accuracies = []

print("开始训练...")
print("-" * 50)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.view(x.shape[0], -1).to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # 计算准确率
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        total += y.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    
    print(f"Epoch [{epoch+1:2d}/{epochs}] - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.2f}%")

print("-" * 50)
print("训练完成!")

## 5. 绘制训练曲线

In [ ]:
# 绘制损失和准确率曲线
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
ax1.plot(range(1, epochs+1), train_losses, marker='o', color='red', linewidth=2)
ax1.set_title('Training Loss Curve')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True, alpha=0.3)

# 准确率曲线
ax2.plot(range(1, epochs+1), train_accuracies, marker='s', color='blue', linewidth=2)
ax2.set_title('Training Accuracy Curve')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('res/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. 模型测试

In [ ]:
model.eval()
correct = 0
test_loss = 0
y_true = []
y_pred = []

print("开始测试...")

with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
        x, y = x.view(x.shape[0], -1).to(device), y.to(device)
        output = model(x)
        test_loss += F.cross_entropy(output, y)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()

        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())

test_loss = test_loss / (batch_idx + 1)
acc = 100. * correct / len(test_loader.dataset)

print("-" * 50)
print('测试结果:')
print(f'测试损失: {test_loss:.4f}')
print(f'测试准确率: {acc:.2f}%')
print(f'正确样本数: {correct}/{len(test_loader.dataset)}')
print("-" * 50)

## 7. 生成混淆矩阵

In [ ]:
# 绘制混淆矩阵
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))

plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title(f'MNIST Confusion Matrix (Accuracy: {acc:.2f}%)')
plt.tight_layout()
plt.savefig('res/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. 可视化预测结果

In [ ]:
# 显示一些测试样本的预测结果
def show_predictions(loader, model, device, num_samples=12):
    model.eval()
    data_iter = iter(loader)
    images, labels = next(data_iter)
    
    # 选择前num_samples个样本
    sample_images = images[:num_samples]
    sample_labels = labels[:num_samples]
    
    # 预测
    with torch.no_grad():
        outputs = model(sample_images.view(num_samples, -1).to(device))
        _, predicted = torch.max(outputs, 1)
    
    # 可视化
    plt.figure(figsize=(12, 8))
    for i in range(num_samples):
        plt.subplot(3, 4, i+1)
        plt.imshow(sample_images[i].squeeze(), cmap='gray')
        
        # 根据预测是否正确设置标题颜色
        color = 'green' if predicted[i] == sample_labels[i] else 'red'
        plt.title(f'True: {sample_labels[i]}, Pred: {predicted[i]}', color=color)
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('res/predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

print("测试样本预测结果:")
show_predictions(test_loader, model, device)

## 9. 保存模型和总结

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'res/mnist_mlp_model.pth')

print("=" * 60)
print("实验总结:")
print(f"模型已保存: res/mnist_mlp_model.pth")
print(f"训练曲线已保存: res/training_curves.png")
print(f"混淆矩阵已保存: res/confusion_matrix.png")
print(f"预测示例已保存: res/predictions.png")
print(f"最终测试准确率: {acc:.2f}%")
print("=" * 60)